# Read in PR full data (including source file in each pull request)

In [ ]:
import pandas as pd

pr_full_data_0 = pd.read_json(
    "pr_full_data_shard_0.jsonl",
    lines=True
)

pr_full_data_1 = pd.read_json(
    "pr_full_data_shard_1.jsonl",
    lines=True
)

pr_full_data_2 = pd.read_json(
    "pr_full_data_shard_2.jsonl",
    lines=True
)

pr_full_data_3 = pd.read_json(
    "pr_full_data_shard_3.jsonl",
    lines=True
)

pr_full_data_4 = pd.read_json(
    "pr_full_data_shard_4.jsonl",
    lines=True
)

pr_full_data_5 = pd.read_json(
    "pr_full_data_shard_5.jsonl",
    lines=True
)

pr_full_data_6 = pd.read_json(
    "pr_full_data_shard_6.jsonl",
    lines=True
)

pr_full_data_7 = pd.read_json(
    "pr_full_data_shard_7.jsonl",
    lines=True
)


## handle errors

In [ ]:
error_df0 = pr_full_data_0[pr_full_data_0['error'].notna()]
error_df0['error'].value_counts()

In [ ]:
error_df1 = pr_full_data_1[pr_full_data_1['error'].notna()]
error_df1['error'].value_counts()

In [ ]:
error_df2 = pr_full_data_2[pr_full_data_2['error'].notna()]
error_df2['error'].value_counts()

In [ ]:
error_df3 = pr_full_data_3[pr_full_data_3['error'].notna()]
error_df3['error'].value_counts()

In [ ]:
error_df4 = pr_full_data_4[pr_full_data_4['error'].notna()]
error_df4['error'].value_counts()

In [ ]:
error_df5 = pr_full_data_5[pr_full_data_5['error'].notna()]
error_df5['error'].value_counts()

In [ ]:
error_df6 = pr_full_data_6[pr_full_data_6['error'].notna()]
error_df6['error'].value_counts()

In [ ]:
error_df7 = pr_full_data_7[pr_full_data_7['error'].notna()]
error_df7['error'].value_counts()

In [ ]:
pr_full_data = pd.concat(
    [
        pr_full_data_0,
        pr_full_data_1,
        pr_full_data_2,
        pr_full_data_3,
        pr_full_data_4,
        pr_full_data_5,
        pr_full_data_6,
        pr_full_data_7,
    ],
    axis=0,
    ignore_index=True
)

error_full_data = pd.concat(
    [
        error_df0,
        error_df1,
        error_df2,
        error_df3,
        error_df4,
        error_df5,
        error_df6,
        error_df7,

    ],
    axis=0,
    ignore_index=True
)

In [ ]:
error_full_data
# there are 121 pull request is not able to access due to deletion and access blocked reason

In [ ]:
pr_error_df = pr_full_data[pr_full_data['error'].notna()].copy()
KEY_COLS = ['html_url']
pr_error_keys = pr_error_df[KEY_COLS]
error_keys = error_full_data[KEY_COLS]

diff_1 = pr_error_keys.merge(
    error_keys,
    on=KEY_COLS,
    how='left',
    indicator=True
).query('_merge == "left_only"')

diff_2 = error_keys.merge(
    pr_error_keys,
    on=KEY_COLS,
    how='left',
    indicator=True
).query('_merge == "left_only"')

len(diff_1), len(diff_2)

# no difference, means we are able to delete the error rows in the pr_full_data dataframe

In [ ]:
pr_full_data_clean = pr_full_data[pr_full_data['error'].isna()].copy()
pr_full_data_clean

In [ ]:
pr_full_data_clean['files'][0]

# Process Repo level data

In [ ]:
pr_full_data_clean.head()

## Remove access blocked Github Repo and remove deleted pull request

In [ ]:

# TODO: remove 451 repo access block repo from merged_pr_df, use html_url match the repo_url then remove
# TODO: remove 404 pull requests, use html_url directly
# TODO:

In [ ]:
import pandas as pd
merged_pr_df = pd.read_parquet("merged_pr_df.parquet")

In [ ]:
display(merged_pr_df)

# group by repo and then sort all the pull request by merged_at, then we apply codeQL check on each pull request full codebase.
# then we should be able to see, after x numbers of pull requests, how many issues are still there? increased? decreased?
# run all the language that codeQL is supporting

In [ ]:
# remove access blocked html url
err451 = error_full_data[
    error_full_data["error"].astype(str).str.contains(r"\b451\b", na=False)
].copy()

blocked_pr_urls = set(err451["html_url"])

blocked_repos = (
    merged_pr_df.loc[merged_pr_df["html_url"].isin(blocked_pr_urls), "repo_url"]
)

merged_pr_df_clean = merged_pr_df[~merged_pr_df["repo_url"].isin(blocked_repos)].copy()
display(merged_pr_df_clean)

In [ ]:
# remove 404 not found pull request

err404 = error_full_data[
    error_full_data["error"]
    .astype(str)
    .str.contains(r"\b404\b", na=False)
].copy()

err404_pr_urls = set(err404["html_url"])

merged_pr_df_clean_2 = merged_pr_df_clean[
    ~merged_pr_df_clean["html_url"].isin(err404_pr_urls)
].copy()


len(merged_pr_df), len(merged_pr_df_clean), len(merged_pr_df_clean_2)

In [ ]:
merged_pr_df_clean_2['repo_url'].value_counts()
# we have 1765 repositories in the original merged_pr_df
# after we remove the 404 pull request and 451 repo
# we still have 1752 repositories

In [ ]:
merged_pr_df_clean_2['repo_url'].value_counts()[lambda x: x >= 1]
# we have xxxx repos can support this study

In [ ]:
repos_gt_1 = (
    merged_pr_df_clean_2['repo_url']
    .value_counts()
    .loc[lambda x: x >= 1] # repo contains at least one pull request
    .index
)

df_multi_repo = merged_pr_df_clean_2[
    merged_pr_df_clean_2['repo_url'].isin(repos_gt_1)
].copy()

df_multi_repo['merged_at'] = pd.to_datetime(df_multi_repo['merged_at'])

df_multi_repo = df_multi_repo.sort_values(
    by=['repo_url', 'merged_at'],
    ascending=[True, True]
)

df_multi_repo['pr_order_in_repo'] = (
    df_multi_repo
    .groupby('repo_url')
    .cumcount()
    + 1
)



In [ ]:
df_multi_repo

# we have 23893 pull requests now

In [ ]:
repo_pr_urls = (
    df_multi_repo
    .groupby('repo_url')['html_url']
    .apply(list)
)

repo_pr_urls_dict = repo_pr_urls.to_dict()

repo_pr_records = (
    df_multi_repo
    .groupby('repo_url')
    .apply(
        lambda g: g[[
            'html_url',
            'merged_at',
            'pr_order_in_repo'
        ]].to_dict(orient='records')
    )
    .to_dict()
)

display(repo_pr_records)

In [ ]:
# we then put the merge commit sha and html url together
url_to_sha = (
    pr_full_data_clean[['html_url', 'merge_commit_sha']]
    .dropna(subset=['html_url', 'merge_commit_sha'])
    .drop_duplicates(subset=['html_url'])
    .set_index('html_url')['merge_commit_sha']
    .to_dict()
)

print(url_to_sha['https://github.com/0x80/isolate-package/pull/132'])


missing_sha = []

for repo_url, pr_list in repo_pr_records.items():
    for pr in pr_list:
        html = pr['html_url']
        sha = url_to_sha.get(html)

        pr['merge_commit_sha'] = sha

        if sha is None:
            missing_sha.append(html)

In [ ]:
all_have_sha = all(
    pr.get('merge_commit_sha') is not None
    for repo_prs in repo_pr_records.values()
    for pr in repo_prs
)

all_have_sha
# all the pull requests have their own merged commit sha, we are able to continue

In [ ]:
repo_pr_records.keys()

In [ ]:
repo_pr_records['https://api.github.com/repos/3rdIteration/btcrecover']

In [ ]:
url_to_files = (
    pr_full_data_clean[['html_url', 'files']]
    .dropna(subset=['html_url'])
    .drop_duplicates(subset=['html_url'])
    .set_index('html_url')['files']
    .to_dict()
)

missing_files = []

for repo_url, pr_list in repo_pr_records.items():
    for pr in pr_list:
        html = pr['html_url']
        files = url_to_files.get(html)

        pr['files'] = files

        if files is None:
            missing_files.append(html)

len(missing_files)

# put files changed in the repo_pr_records, we can distinguish the files changed by agent by each pull request

In [ ]:
# check data
repo_pr_records['https://api.github.com/repos/3rdIteration/btcrecover'][0]['files']

In [ ]:
from pathlib import PurePosixPath

# from codeql document: https://codeql.github.com/docs/codeql-overview/supported-languages-and-frameworks/
# codeql only support these languages

CODEQL_LANGUAGES = {
    "C/C++",
    "C#",
    "Go",
    "Java/Kotlin",
    "JavaScript",
    "TypeScript",
    "Python",
    "Ruby",
    "Rust",
    "Swift",
    "GitHub Actions",
}

JS_EXTENSIONS = {
    ".js", ".jsx", ".mjs", ".es", ".es6",
    ".htm", ".html", ".xhtm", ".xhtml",
    ".vue",
    ".hbs", ".ejs", ".njk",
    ".json",
    ".yaml", ".yml",
    ".raml",
    ".xml",
    ".ts", ".tsx", ".mts", ".cts"
}

# TS_EXTENSIONS = {
#     ".ts", ".tsx", ".mts", ".cts"
# }
# ts goes into js as codeql does it

CSHARP_EXTENSIONS = {
    ".sln",
    ".slnx",
    ".csproj",
    ".cs",
    ".cshtml",
    ".xaml",
}


RUBY_EXTENSIONS = {
    ".rb",        # Ruby source
    ".erb",       # Embedded Ruby (Rails views)
    ".gemspec",   # Gem specification
}

RUBY_FILENAMES = {
    "Gemfile",
}

RUST_EXTENSIONS = {
    ".rs",        # Rust source
}

RUST_FILENAMES = {
    "Cargo.toml", # Rust package manifest
}


def codeql_language_from_filename(filename: str) -> str:
    p = PurePosixPath(filename)
    ext = p.suffix.lower()
    name = p.name
    path = p.as_posix()

    # github actions
    if ".github/workflows" in path and ext in {".yml", ".yaml"}:
        return "Other"

    # JavaScript and Typescript
    if ext in JS_EXTENSIONS:
        return "JavaScript/TypeScript"
    
    # Python
    if ext in {".py", ".pyi"}:
        return "Python"

    # Java / Kotlin
    if ext in {".java", ".kt"}:
        return "Java/Kotlin"

    # C / C++
    if ext in {".c", ".h", ".cc", ".cpp", ".cxx", ".hpp"}:
        return "C/C++"

    # C#
    if ext in CSHARP_EXTENSIONS:
        return "C#"
    
    # Ruby
    if ext in RUBY_EXTENSIONS or name in RUBY_FILENAMES:
        return "Ruby"

    # Rust
    if ext in RUST_EXTENSIONS or name in RUST_FILENAMES:
        return "Rust"

    # # Go
    # if ext == ".go":
    #     return "Go"


    return "Other"

def extract_codeql_languages_from_pr(files: list[dict]) -> list[str]:
    return sorted({
        codeql_language_from_filename(f["filename"])
        for f in files or []
        if "filename" in f
    })

def apply_codeql_languages_to_all_prs(repo_pr_records: dict) -> dict:
    """
    Mutates repo_pr_records in-place.
    For each PR, adds a `language` field:
        pr["language"] = list of CodeQL languages
    """
    for repo_url, pr_list in repo_pr_records.items():
        for pr in pr_list:
            files = pr.get("files", [])
            pr["language"] = extract_codeql_languages_from_pr(files)
    return repo_pr_records


In [ ]:
repo_pr_records = apply_codeql_languages_to_all_prs(repo_pr_records)
# get languages for each pull request

In [ ]:
repo_pr_records['https://api.github.com/repos/0x4D31/galah'][0]

In [ ]:
def get_all_languages(repo_pr_records: dict) -> set[str]:
    all_languages = set()

    for pr_list in repo_pr_records.values():
        for pr in pr_list:
            for lang in pr.get("language", []):
                all_languages.add(lang)

    return all_languages

all_languages = get_all_languages(repo_pr_records)

print(len(all_languages))
print(sorted(all_languages))
# there are two languages appeared in the pull requests

In [ ]:
def count_only_other_prs(repo_pr_records: dict) -> int:
    count = 0

    for pr_list in repo_pr_records.values():
        for pr in pr_list:
            langs = pr.get("language", [])
            if langs and all(lang == "Other" for lang in langs):
                count += 1

    return count

# now we need to check the pull request only contains other language, we need to remove them

only_other_pr_count = count_only_other_prs(repo_pr_records)
print("PRs with only 'Other':", only_other_pr_count)

In [ ]:
def remove_only_other_prs(repo_pr_records: dict) -> dict:
    cleaned = {}

    for repo_url, pr_list in repo_pr_records.items():
        new_pr_list = []

        for pr in pr_list:
            langs = pr.get("language", [])
            if not (langs and all(lang == "Other" for lang in langs)):
                new_pr_list.append(pr)

        if new_pr_list:
            cleaned[repo_url] = new_pr_list

    return cleaned

repo_pr_records_clean = remove_only_other_prs(repo_pr_records)

print("Before cleanup:")
print("Repos:", len(repo_pr_records))
print("PRs:", sum(len(v) for v in repo_pr_records.values()))
print("Only-Other PRs:", count_only_other_prs(repo_pr_records))

repo_pr_records = remove_only_other_prs(repo_pr_records)

print("\nAfter cleanup:")
print("Repos:", len(repo_pr_records))
print("PRs:", sum(len(v) for v in repo_pr_records.values()))
print("Only-Other PRs:", count_only_other_prs(repo_pr_records))

# after we remove the other only pull request, we have  repos remain

In [ ]:
# import os
# import json
# import pandas as pd
# from datetime import datetime

# os.makedirs("artifacts", exist_ok=True)
# output_path = "artifacts/repo_pr_records_clean.json"

# def json_serializer(obj):
#     if isinstance(obj, (pd.Timestamp, datetime)):
#         return obj.isoformat()
#     return str(obj)

# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(
#         repo_pr_records_clean,
#         f,
#         indent=2,
#         ensure_ascii=False,
#         default=json_serializer
#     )

# print(f"✅ Saved to {output_path}")
# this file is 1.5 GB

In [ ]:
repo_pr_records_clean['https://api.github.com/repos/XiangpengHao/liquid-cache']

In [ ]:
# now we have cleaned the data and all the repos and pull request within valid repos
# next we should be able to run codeql on the entire codebase for that specific merged commit and then we check wether the issues are introduced by agent

# we only analyse the language in the pull request's language list, in this way we have a high chance to find wether the issue is introduced by agent 
# and faster for the experiment

In [ ]:
# I'm trying apply codeQL on the each pull request for the repo

# step1: extract data
def extract_repo_records(api_url, pr_list):

    repo_url = api_url.replace("https://api.github.com", "https://github.com")
    if "/repos/" in repo_url:
        repo_url = repo_url.replace("/repos/", "/")
        
    records = []
    for item in pr_list:
        print(item)
        records.append({
            'html_url': item['html_url'],
            'merge_commit_sha': item['merge_commit_sha'],
            'pr_order': item['pr_order_in_repo'],
            'language': item['language']
        })
    return repo_url, records

In [ ]:
CODEQL_QUERY_SUITES = {
    "C/C++": "cpp-security-and-quality.qls",
    "C#": "csharp-security-and-quality.qls",
    "Java/Kotlin": "java-security-and-quality.qls",
    "JavaScript/TypeScript": "javascript-security-and-quality.qls",
    "Python": "python-security-and-quality.qls",
    "Ruby": "ruby-security-and-quality.qls",
    "Rust": "rust-security-and-quality.qls",
}

CODEQL_CLI_LANG_MAP = {
    "C/C++": "cpp",
    "C#": "csharp",
    "Java/Kotlin": "java",
    "JavaScript/TypeScript": "javascript",
    "Python": "python",
    "Ruby": "ruby",
    "Rust": "rust",
}

CODEQL_BUILD_MODE = {
    "C/C++": "none",
    "C#": "none",
    "Java/Kotlin": "none",
    "JavaScript/TypeScript": "none",
    "Python": "none",
    "Ruby": "none",
    "Rust": "none",
}

def select_build_mode(pr_languages: list[str]) -> str:
    for lang in pr_languages:
        if CODEQL_BUILD_MODE.get(lang) == "none":
            return "none"
    return "autobuild"



In [ ]:
def run_scan_pipeline(repo_url, records):
    import subprocess
    import os
    import shutil

    CODEQL_BIN = "/Users/guanhao/codeql-home/codeql/codeql"
    BASE_REPORT_DIR = "codeql-reports"

    parts = repo_url.rstrip("/").split("/")
    repo_folder_name = f"{parts[-2]}_{parts[-1]}"

    for pr in records:
        sha = pr["merge_commit_sha"]
        pr_id = pr["pr_order"]
        pr_languages = pr.get("language", [])

        pr_langs = [
            l for l in pr_languages
            if l in CODEQL_CLI_LANG_MAP and l != "GitHub Actions"
        ]

        if not pr_langs:
            print(f"\n⏭️  [PR {pr_id}] Skipped (no CodeQL-supported languages)")
            continue

        print(f"\n🚀 [PR {pr_id}] SHA: {sha}")
        print(f"   🧠 PR languages (CodeQL): {pr_langs}")

        report_dir = os.path.join(
            BASE_REPORT_DIR, repo_folder_name, f"PR_{pr_id}"
        )
        os.makedirs(report_dir, exist_ok=True)

        source_dir = f"temp_source_pr_{pr_id}"
        os.makedirs(source_dir, exist_ok=True)

        try:
            # ===== checkout merge commit (once per PR) =====
            subprocess.run(
                f"git init && "
                f"git remote add origin {repo_url} && "
                f"git fetch --depth 1 origin {sha} && "
                f"git checkout FETCH_HEAD",
                shell=True,
                cwd=source_dir,
                check=True,
                capture_output=True,
            )

            # ===== language-level isolation =====
            for pr_lang in pr_langs:
                cli_lang = CODEQL_CLI_LANG_MAP[pr_lang]
                query_suite = CODEQL_QUERY_SUITES[pr_lang]

                db_dir = f"temp_db_pr_{pr_id}_{cli_lang}"
                db_dir_abs = os.path.abspath(db_dir)

                print(f"\n   🏗️  [{pr_lang}] Creating DB ({cli_lang})")

                build_mode = CODEQL_BUILD_MODE.get(pr_lang, "none")

                try:
                    # --- DB create ---
                    subprocess.run(
                        f"{CODEQL_BIN} database create {db_dir_abs} "
                        f"--source-root=. "
                        f"--language={cli_lang} "
                        f"--build-mode={build_mode}",
                        shell=True,
                        cwd=source_dir,
                        check=True,
                    )

                    output_path = os.path.join(
                        report_dir,
                        f"{cli_lang}_security_report.sarif"
                    )

                    print(f"   🔍 [{pr_lang}] Analyzing...")

                    # --- analyze ---
                    subprocess.run(
                        f"{CODEQL_BIN} database analyze "
                        f"{db_dir_abs} "
                        f"{query_suite} "
                        f"--format=sarif-latest "
                        f"--output={os.path.abspath(output_path)}",
                        shell=True,
                        check=True,
                    )

                    print(f"   ✅ [{pr_lang}] Report: {output_path}")

                except Exception as lang_err:
                    # ⚠️ language-level failure is tolerated
                    print(
                        f"   ⚠️  [{pr_lang}] Failed, skipped.\n"
                        f"       Reason: {lang_err}"
                    )

                finally:
                    # --- always clean DB dir ---
                    if os.path.exists(db_dir_abs):
                        shutil.rmtree(db_dir_abs)

        except Exception as pr_err:
            # ❌ only catastrophic PR-level failures land here
            print(f"   ❌ Error preparing PR {pr_id}: {pr_err}")

        finally:
            print("   🧹 Cleaning up source workspace...")
            if os.path.exists(source_dir):
                shutil.rmtree(source_dir)

    print(f"\n✨ All scans for {repo_folder_name} completed.")

In [ ]:
# tested: rust, java\kotlin, c\c++, python, js\ts, c#, ruby
# we can use these languages

# full pipeline get codeql report

import os
import traceback

BASE_REPORT_DIR = "codeql-reports"

os.makedirs(BASE_REPORT_DIR, exist_ok=True)

total = len(repo_pr_records_clean)
done = 0
skipped = 0
failed = 0

for idx, (api_repo_url, pr_records) in enumerate(repo_pr_records_clean.items(), start=1):

    print(f"\n{'='*60}")
    print(f"📦 [{idx}/{total}] Processing repo:")
    print(f"    {api_repo_url}")

    # ===== API url → GitHub url & records =====
    repo_url, records = extract_repo_records(api_repo_url, pr_records)

    parts = repo_url.rstrip("/").split("/")
    repo_folder_name = f"{parts[-2]}_{parts[-1]}"
    repo_report_path = os.path.join(BASE_REPORT_DIR, repo_folder_name)
    done_marker = os.path.join(repo_report_path, "_DONE")

    # ===== skip if fully completed =====
    if os.path.exists(done_marker):
        print(f"⏭️  Skipped (already completed): {repo_folder_name}")
        skipped += 1
        continue

    print(f"🚀 Running CodeQL for {repo_folder_name}")
    os.makedirs(repo_report_path, exist_ok=True)

    try:
        run_scan_pipeline(repo_url, records)

        # ===== mark repo as completed =====
        with open(done_marker, "w") as f:
            f.write("ok\n")

        print(f"✅ Completed: {repo_folder_name}")
        done += 1

    except Exception as e:
        failed += 1
        print(f"❌ Failed: {repo_folder_name}")
        print("   Reason:", str(e))
        traceback.print_exc()

print(f"\n{'='*60}")
print("🎉 Batch finished")
print(f"✅ Completed repos : {done}")
print(f"⏭️  Skipped repos   : {skipped}")
print(f"❌ Failed repos    : {failed}")
print(f"📦 Total repos     : {total}")

In [ ]:
# good job Hao!

In [ ]:
# ========= imports =========
import os
import json
import glob
import subprocess
import shutil
import re
from collections import defaultdict
from urllib.parse import urlparse


# ========= SARIF parsing =========
def parse_sarif_findings(sarif_path: str) -> list[dict]:
    with open(sarif_path, "r", encoding="utf-8") as f:
        sarif = json.load(f)

    findings = []
    for run in sarif.get("runs", []):
        for r in run.get("results", []) or []:
            rule_id = r.get("ruleId") or (r.get("rule") or {}).get("id")
            message = (r.get("message") or {}).get("text", "")
            level = r.get("level")

            locs = r.get("locations") or []
            if not locs:
                continue

            pl = locs[0].get("physicalLocation") or {}
            artifact = (pl.get("artifactLocation") or {}).get("uri")
            region = pl.get("region") or {}
            start_line = region.get("startLine")

            if not artifact or not start_line:
                continue

            findings.append({
                "rule_id": rule_id,
                "message": message,
                "level": level,
                "file": artifact,
                "start_line": int(start_line),
            })

    return findings


# ========= path normalization =========
def normalize_sarif_path(uri: str, repo_root_abs: str) -> str:
    if uri.startswith("file://"):
        path = urlparse(uri).path
    else:
        path = uri

    path = os.path.normpath(path).lstrip("./")

    if os.path.isabs(path):
        repo_root_abs = os.path.normpath(repo_root_abs)
        if path.startswith(repo_root_abs):
            path = os.path.relpath(path, repo_root_abs)

    return path.lstrip("./")


# ========= introduced-by-PR marking =========
def mark_introduced(findings, added_lines_by_file, repo_root_abs):
    out = []
    for f in findings:
        file_path = normalize_sarif_path(f["file"], repo_root_abs)
        line = f["start_line"]

        introduced = line in added_lines_by_file.get(file_path, set())

        f2 = dict(f)
        f2["normalized_file"] = file_path
        f2["introduced_by_pr"] = introduced
        out.append(f2)
    return out


# ========= diff parsing =========
HUNK_RE = re.compile(r"^\@\@ -\d+(?:,\d+)? \+(\d+)(?:,(\d+))? \@\@")

def get_added_lines_by_file(repo_dir: str, merge_sha: str) -> dict[str, set[int]]:
    cmd = ["git", "diff", "-U0", f"{merge_sha}^1", merge_sha]
    p = subprocess.run(
        cmd,
        cwd=repo_dir,
        capture_output=True,
        text=True,
        check=True
    )

    added = defaultdict(set)
    current_file = None
    new_line = None

    for line in p.stdout.splitlines():
        if line.startswith("+++ b/"):
            current_file = line[len("+++ b/"):].strip().lstrip("./")
            new_line = None
            continue

        m = HUNK_RE.match(line)
        if m:
            new_line = int(m.group(1))
            continue

        if current_file is None or new_line is None:
            continue

        if line.startswith("+") and not line.startswith("+++"):
            added[current_file].add(new_line)
            new_line += 1
        elif line.startswith("-") and not line.startswith("---"):
            continue
        else:
            new_line += 1

    return dict(added)

def api_to_clone_url(api_url: str) -> str:
    return api_url.replace(
        "https://api.github.com/repos/",
        "https://github.com/"
    )


def repo_folder_to_api_url(repo_folder: str) -> str:
    # BOINC_boinc -> https://api.github.com/repos/BOINC/boinc
    owner, repo = repo_folder.split("_", 1)
    return f"https://api.github.com/repos/{owner}/{repo}"

def write_jsonl(path: str, rows: list[dict]):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
            



In [ ]:
# BASE_REPORTS = "codeql-reports"
# OUTPUT_BASE = "issues-introduced-by-agent"
# TMP_REPO = "_tmp_repo"

# os.makedirs(OUTPUT_BASE, exist_ok=True)

# for repo_folder in sorted(os.listdir(BASE_REPORTS)):
#     repo_path = os.path.join(BASE_REPORTS, repo_folder)
#     if not os.path.isdir(repo_path):
#         continue

#     print(f"\n📦 Repo: {repo_folder}")

#     api_repo = repo_folder_to_api_url(repo_folder)
#     if api_repo not in repo_pr_records_clean:
#         print("  ⚠️ Repo not found in records, skipped")
#         continue

#     clone_url = api_to_clone_url(api_repo)
#     pr_records = repo_pr_records_clean[api_repo]

#     # PR order -> record
#     pr_by_order = {
#         p["pr_order_in_repo"]: p
#         for p in pr_records
#         if p.get("merge_commit_sha")
#     }

#     for pr_dir in sorted(os.listdir(repo_path)):
#         if not pr_dir.startswith("PR_"):
#             continue

#         pr_order = int(pr_dir.replace("PR_", ""))
#         print(f"  🔹 PR {pr_order}")

#         if pr_order not in pr_by_order:
#             print("    ⚠️ PR not in records, skipped")
#             continue

#         pr_record = pr_by_order[pr_order]
#         merge_sha = pr_record["merge_commit_sha"]

#         sarif_dir = os.path.join(repo_path, pr_dir)
#         sarif_files = glob.glob(os.path.join(sarif_dir, "*.sarif"))
#         if not sarif_files:
#             print("    ⚠️ No SARIF files, skipped")
#             continue

#         # ----- temp repo -----
#         if os.path.exists(TMP_REPO):
#             shutil.rmtree(TMP_REPO)
#         os.makedirs(TMP_REPO, exist_ok=True)

#         try:
#             # checkout
#             subprocess.run(
#                 f"git init && "
#                 f"git remote add origin {clone_url} && "
#                 f"git fetch --depth 2 origin {merge_sha} && "
#                 f"git checkout {merge_sha}",
#                 shell=True,
#                 cwd=TMP_REPO,
#                 check=True,
#                 stdout=subprocess.DEVNULL,
#                 stderr=subprocess.DEVNULL,
#             )

#             # ensure parent
#             subprocess.run(
#                 ["git", "rev-parse", f"{merge_sha}^1"],
#                 cwd=TMP_REPO,
#                 check=True,
#                 stdout=subprocess.DEVNULL,
#             )

#             # diff
#             added_lines = get_added_lines_by_file(TMP_REPO, merge_sha)

#             all_marked = []
#             all_introduced = []

#             for sarif_path in sarif_files:
#                 findings = parse_sarif_findings(sarif_path)
#                 marked = mark_introduced(
#                     findings,
#                     added_lines,
#                     os.path.abspath(TMP_REPO)
#                 )
#                 all_marked.extend(marked)
#                 all_introduced.extend(
#                     f for f in marked if f["introduced_by_pr"]
#                 )

#             # ----- save -----
#             out_dir = os.path.join(OUTPUT_BASE, repo_folder, pr_dir)
#             os.makedirs(out_dir, exist_ok=True)

#             write_jsonl(os.path.join(out_dir, "findings_all.jsonl"), all_marked)
#             write_jsonl(os.path.join(out_dir, "findings_introduced.jsonl"), all_introduced)

#             summary = {
#                 "repo": api_repo,
#                 "repo_folder": repo_folder,
#                 "pr_order_in_repo": pr_order,
#                 "pr_html_url": pr_record["html_url"],
#                 "merge_sha": merge_sha,
#                 "sarif_files": [os.path.basename(p) for p in sarif_files],
#                 "total_findings": len(all_marked),
#                 "introduced_findings": len(all_introduced),
#                 "pr_author_type": "agent",
#             }

#             with open(os.path.join(out_dir, "summary.json"), "w") as f:
#                 json.dump(summary, f, indent=2)

#             print(f"    ✅ Saved ({len(all_introduced)} introduced)")

#         except Exception as e:
#             print(f"    ❌ Error: {e}")

#         finally:
#             if os.path.exists(TMP_REPO):
#                 shutil.rmtree(TMP_REPO)

In [ ]:
# good job! Hao!

In [ ]:
# # full pipeline checkout issues actually introduced by agent

# repo_pr_records_clean['https://api.github.com/repos/OpenHFT/Chronicle-Core']